## Important points:

#### In chaining, we create "array of LinkedLists" which means; every element of the array is a LL object.

#### Each element of the array is called BUCKET and whole array is called BUCKETS.

#### Also, each node of this LL will contain three elements: key, value, next.

In [1]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

In [2]:
class LinkedList:
    def __init__(self):
        # creaing an empty LL (It means the value of head of LL should be None.)
        self.head = None
        # counting no of nodes in LL
        self.n = 0
    
    
    def __len__(self):
        return self.n
      
        
    # Using magic method    
    def __str__(self):
        curr = self.head
        result = ''
        
        while curr != None:
            result = result + curr.key  + ':' + str(curr.value) + ", "
            curr = curr.next
        
        return result[:-2]
    
    
    def append(self, key, value):
        # creating a new node
        new_node = Node(key, value)
        # checking for empty list
        if self.head == None:
            self.head = new_node
            self.n = self.n + 1
            return 
        
        curr = self.head
        # traversing till one element before the last element
        while curr.next != None:
            curr = curr.next
        
        # Now we our "curr" is the tail, so
        curr.next = new_node
        # increment n
        self.n = self.n + 1
        
    
    def remove(self, key):
        # checking if its an empty LL
        if self.head == None:
            return 'Empty LL'
        
        # If LL has only one node and its key is equal to given key
        if self.head.key == key:
            return self.delete_head()
        
        curr = self.head
        # Here, we need to check the key of curr.next, if it's equal to key or not; as
        # we cannot go back in LL and we want to stop one before the node whose key is equal
        # to the given key
        while curr.next != None:
            if curr.next.key == key:
                break
            curr = curr.next
            
        """
        Again there are two cases even after exiting from the loop: 
        1. if there is a data which matches with value 
        2. or none of the data matched with the given value
        """
        if curr.next == None:
            return "Item Not Found!"
        else:
            curr.next = curr.next.next
            self.n = self.n - 1
    
    
    def delete_head(self):
        # checking if its an empty LL
        if self.head == None:
            return "Empty LL"
        
        self.head = self.head.next
        self.n = self.n -1
      
    
    def search(self, key):
        curr = self.head
        pos = 0
        
        while curr != None:
            if curr.key == key:
                return pos
            curr = curr.next
            pos += 1
        return -1
    
    
    # Using magic method as we need to find the value using indexing
    def __getitem__(self, idx):
        curr = self.head
        pos = 0
        
        while curr != None:
            if pos == idx:
                return curr.data
            curr = curr.next
            pos += 1
        return "IndexError"
    
    def get_node_at_index(self, node_idx):
        temp = self.head
        counter = 0
        
        while temp is not None:
            if counter == node_idx:
                return temp
            
            temp = temp.next
            counter += 1

### Hashing without Load Factor

In [3]:
class Dictionary:
    def __init__(self, capacity):
        # total no of bucket 
        self.capacity = capacity
        # num of key value pairs present in dictionary
        self.size = 0
        # creating an array of LL
        self.buckets = self.make_array(self.capacity)
    
    def make_array(self, capacity):
        buckets = []
        for i in range(capacity):
            buckets.append(LinkedList())
        return buckets
    
    def put(self, key, value):
        # first finding the index of bucket (LL object) where we need to insert
        # Note, in chaining the index of bucket where we need to insert is fixed
        bucket_index = self.hash_func(key)
        
        """
        Here, there will be two cases:
          1. if the key is already present (we simply need to update its value)
          2. if the key is not present (we have to insert key value pair from tail)
        
        """ 
        # So, first we will find node index in that bucket(LL object) by appling search method 
        node_index = self.buckets[bucket_index].search(key)
        
        if node_index == -1:
            # inserting from tail
            self.buckets[bucket_index].append(key, value)
            self.size += 1
        else:
            # updating the value of key
            node = self.buckets[bucket_index].get_node_at_index(node_index)
            node.value = value
    
    def hash_func(self, key):
        return abs(hash(key)) % self.capacity

In [4]:
d1 = Dictionary(4)

In [5]:
d1.put("Ck", 1500)

In [6]:
d1.buckets

In [12]:
print(d1.buckets[2])

Ck:1500


In [13]:
d1.put("Md", 1501)

In [15]:
print(d1.buckets[0])

Md:1501


In [16]:
d1.put("Mh", 1499)

In [18]:
print(d1.buckets[0])

Md:1501, Mh:1499


In [19]:
d1.put("Mh", "I got changed :(")

In [20]:
print(d1.buckets[0])

Md:1501, Mh:I got changed :(


### Hashing with Load Factor

#### """Load Factor is also called Lambda"""
The main changes are applied when instering an key value pair (every time load factor is checked using a threshold)

In [ ]:
class Dictionary:
    def __init__(self, capacity):
        # total no of bucket 
        self.capacity = capacity
        # num of key value pairs present in dictionary
        self.size = 0
        # creating an array of LL
        self.buckets = self.make_array(self.capacity)
    
    def make_array(self, capacity):
        buckets = []
        for i in range(capacity):
            buckets.append(LinkedList())
        return buckets
    
    def put(self, key, value):
        # first finding the index of bucket (LL object) where we need to insert
        # Note, in chaining the index of bucket where we need to insert is fixed
        bucket_index = self.hash_func(key)
        
        """
        Here, there will be two cases:
          1. if the key is already present (we simply need to update its value)
          2. if the key is not present (we have to insert key value pair from tail)
        
        """ 
        # So, first we will find node index in that bucket(LL object) by appling search method 
        node_index = self.buckets[bucket_index].search(key)
        
        if node_index == -1:
            # inserting from tail
            self.buckets[bucket_index].append(key, value)
            self.size += 1
            
            # checking load factor creteria (lf = size/capacity)
            if (self.size/self.capacity >= 2):
                self.rehash()
                
        else:
            # updating the value of key
            node = self.buckets[bucket_index].get_node_at_index(node_index)
            node.value = value
    
    def hash_func(self, key):
        return abs(hash(key)) % self.capacity
    
    
    def rehash(self):
        # defining a new capacity and so size
        self.capacity = self.capacity * 2
        self.size = 0
        # storing precious array for acessing later
        old_buckets = self.buckets
        
        # redefining buckets
        self.buckets = self.make_array(self.capacity)
        
        # Now accessing each key-value pair from old buckets and putting them into new buckets
        for bckt in old_buckets:
            for i in range(len(bckt)):
                # fetching node
                curr_node = bckt.get_node_at_index(i)
                # fetching its key and value
                k = curr_node.key
                v = curr_node.value
                
                # finally putting key-value pair into new buckets
                self.put(k, v)